In [1]:
mecab

NameError: name 'mecab' is not defined

In [2]:
import MeCab

text = "吾輩は猫である"
# 形態素解析の結果をChasenの出力方式で表示
t = MeCab.Tagger("-Ochasen")
result = t.parse(text)
print(result)

吾輩	ワガハイ	吾輩	名詞-代名詞-一般		
は	ハ	は	助詞-係助詞		
猫	ネコ	猫	名詞-一般		
で	デ	だ	助動詞	特殊・ダ	連用形
ある	アル	ある	助動詞	五段・ラ行アル	基本形
EOS



In [3]:
# 形態素解析の結果を確認
result

'吾輩\tワガハイ\t吾輩\t名詞-代名詞-一般\t\t\nは\tハ\tは\t助詞-係助詞\t\t\n猫\tネコ\t猫\t名詞-一般\t\t\nで\tデ\tだ\t助動詞\t特殊・ダ\t連用形\nある\tアル\tある\t助動詞\t五段・ラ行アル\t基本形\nEOS\n'

In [4]:
# 形態素解析の結果を、改行を区切りとして行ごとに分割
results = result.splitlines()

# EOSの行は対象外とする
for res in results[:-1]:
    res_split = res.split("\t")
    print(res_split)

['吾輩', 'ワガハイ', '吾輩', '名詞-代名詞-一般', '', '']
['は', 'ハ', 'は', '助詞-係助詞', '', '']
['猫', 'ネコ', '猫', '名詞-一般', '', '']
['で', 'デ', 'だ', '助動詞', '特殊・ダ', '連用形']
['ある', 'アル', 'ある', '助動詞', '五段・ラ行アル', '基本形']


In [10]:
import MeCab

documents = ["子供が走る","車が走る","子供の脇を車が走る"]
words_list = []

#　形態素解析の結果をChasenの出力形式で表示
t = MeCab.Tagger("-Ochasen")

# 各文に形態素解析を実行
for s in documents:
    s_parsed = t.parse(s)
    words_s = []
    
    #　各文の形態素をリストにまとめる
    for line in s_parsed.splitlines()[:-1]:
        words_s.append(line.split("\t")[0])
    words_list.append(words_s)

print(words_list)

[['子供', 'が', '走る'], ['車', 'が', '走る'], ['子供', 'の', '脇', 'を', '車', 'が', '走る']]


In [11]:
# 生成する辞書
word2int = {}
i = 0

# 各文書の単語のリストに対して処理を反復
for words in words_list:
    # 文書内の各単語に対して処理を反復
    for word in words:
        # 単語が辞書に含まれていなければ追加して対応する整数を割り当てる
        if word not in word2int:
            word2int[word] = i
            i += 1

print(word2int)

{'子供': 0, 'が': 1, '走る': 2, '車': 3, 'の': 4, '脇': 5, 'を': 6}


In [13]:
import numpy as np

# BoWを計算し、文書X単語の行列を生成
bow = np.zeros((len(words_list), len(word2int)), dtype=np.int)

# 各行の単語を抽出し、単語の出現回数をカウント
for i,words in enumerate(words_list):
    for word in words:
        bow[i, word2int[word]] += 1

print(bow)

[[1 1 1 0 0 0 0]
 [0 1 1 1 0 0 0]
 [1 1 1 1 1 1 1]]


In [14]:
import pandas as pd

pd.DataFrame(bow, columns=list(word2int))

,子供,が,走る,車,の,脇,を
0,1,1,1,0,0,0,0
1,0,1,1,1,0,0,0
2,1,1,1,1,1,1,1


In [15]:
from gensim import corpora

# 辞書を作成する
word2int_gs = corpora.Dictionary(words_list)
print(word2int_gs)

Dictionary(7 unique tokens: ['が', '子供', '走る', '車', 'の']...)


In [16]:
# 単語と整数の対応
print(word2int_gs.token2id)

{'が': 0, '子供': 1, '走る': 2, '車': 3, 'の': 4, 'を': 5, '脇': 6}


In [17]:
# 1番目の文書に含まれる単語の出現回数をカウント
print(word2int_gs.doc2bow(words_list[0]))

[(0, 1), (1, 1), (2, 1)]


In [19]:
from gensim import matutils

# Bag of Wordsを計算し、文書X単語の行列を生成
bow_gs = np.array([matutils.corpus2dense([word2int_gs.doc2bow(words)], 
                                        num_terms=len(word2int)).T[0] for words in words_list]).astype(np.int)
print(bow_gs)

[[1 1 1 0 0 0 0]
 [1 0 1 1 0 0 0]
 [1 1 1 1 1 1 1]]


In [20]:
# pandasのデータフレームに変換
bow_gs_df = pd.DataFrame(bow_gs, columns=list(word2int_gs.values()))
bow_gs_df

,が,子供,走る,車,の,を,脇
0,1,1,1,0,0,0,0
1,1,0,1,1,0,0,0
2,1,1,1,1,1,1,1


In [21]:
# 単語をスペース区切りで並べた文を生成
words_split = np.array([" ".join(words) for words in words_list])
print(words_split)

['子供 が 走る' '車 が 走る' '子供 の 脇 を 車 が 走る']


In [22]:
from sklearn.feature_extraction.text import CountVectorizer

# Bag of Wordsを計算
vectorizer = CountVectorizer(token_pattern=u"(?u)\\b\\w+\\b")
bow_vec = vectorizer.fit_transform(words_split)

# NumPy配列に変換
bow_vec.toarray()

array([[1, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 1, 1],
       [1, 1, 1, 1, 1, 1, 1]])

In [23]:
vectorizer.get_feature_names()

['が', 'の', 'を', '子供', '脇', '走る', '車']

In [24]:
bow_gs_df

,が,子供,走る,車,の,を,脇
0,1,1,1,0,0,0,0
1,1,0,1,1,0,0,0
2,1,1,1,1,1,1,1


In [25]:
# TFとしてBowを使用
tf = bow_gs
print(tf)

[[1 1 1 0 0 0 0]
 [1 0 1 1 0 0 0]
 [1 1 1 1 1 1 1]]


In [26]:
# IDFを計算
idf = np.log((bow_gs.shape[0] + 1) / (np.sum(bow_gs, axis=0, keepdims=0) + 1))
print(idf)

[0.         0.28768207 0.         0.28768207 0.69314718 0.69314718
 0.69314718]


In [28]:
# TF-IDFを計算
tf_idf = tf * (idf + 1)
tf_idf_normalized = tf_idf / np.sqrt(np.sum(tf_idf**2, axis=1, keepdims=True))
print(tf_idf_normalized)

[[0.52284231 0.67325467 0.52284231 0.         0.         0.
  0.        ]
 [0.52284231 0.         0.52284231 0.67325467 0.         0.
  0.        ]
 [0.26806191 0.34517852 0.26806191 0.34517852 0.45386827 0.45386827
  0.45386827]]


In [29]:
from sklearn.feature_extraction.text import TfidfTransformer

# TfidfTransformerクラスをインスタンス化
tfidf = TfidfTransformer(use_idf=True, norm="l2", smooth_idf=True)

# TF-IDFを算出
print(tfidf.fit_transform(bow_gs).toarray())

[[0.52284231 0.67325467 0.52284231 0.         0.         0.
  0.        ]
 [0.52284231 0.         0.52284231 0.67325467 0.         0.
  0.        ]
 [0.26806191 0.34517852 0.26806191 0.34517852 0.45386827 0.45386827
  0.45386827]]


In [1]:
def fibonacci(n):
    if n == 0:
        return 1
    elif n == 1:
        return 1
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)
    
for i in range(8):
    print(fibonacci(i))

1
1
2
3
5
8
13
21
